In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [4]:
# Hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [5]:
# Training

#Hyperparameter
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
epochs = 10
for i in range(epochs):
    print(f"Epoch {i+1}\n----------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------
loss: 2.293169  [   64/60000]
loss: 2.291357  [ 6464/60000]
loss: 2.266582  [12864/60000]
loss: 2.269413  [19264/60000]
loss: 2.258985  [25664/60000]
loss: 2.207666  [32064/60000]
loss: 2.223298  [38464/60000]
loss: 2.181028  [44864/60000]
loss: 2.181575  [51264/60000]
loss: 2.143875  [57664/60000]
Test Error: 
 Accuracy: 35.4%, Avg loss: 2.145228 

Epoch 2
----------
loss: 2.151960  [   64/60000]
loss: 2.149991  [ 6464/60000]
loss: 2.085181  [12864/60000]
loss: 2.110286  [19264/60000]
loss: 2.058284  [25664/60000]
loss: 1.981329  [32064/60000]
loss: 2.022547  [38464/60000]
loss: 1.932645  [44864/60000]
loss: 1.942911  [51264/60000]
loss: 1.863803  [57664/60000]
Test Error: 
 Accuracy: 55.1%, Avg loss: 1.869580 

Epoch 3
----------
loss: 1.901118  [   64/60000]
loss: 1.876508  [ 6464/60000]
loss: 1.753002  [12864/60000]
loss: 1.801563  [19264/60000]
loss: 1.690371  [25664/60000]
loss: 1.635498  [32064/60000]
loss: 1.667700  [38464/60000]
loss: 1.564315  [44864/60000]

In [7]:
# save model
import torchvision.models as models
torch.save(model, "tutorials.pth")

In [8]:
# load model
import torch
import torchvision.models as models

model = torch.load("tutorials.pth")
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [9]:
#prediction
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to("cpu")
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


Predicted: "Ankle boot", Actual: "Ankle boot"
